In [1]:
import time
import tracemalloc
import csv

In [2]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_global(v, w, pointers):
  i,j = len(v), len(w)
  new_v = []
  new_w = []
  while True:
      di, dj = pointers[i][j]
      if (di,dj) == LEFT:
          new_v.append('-')
          new_w.append(w[j-1])
      elif (di,dj) == UP:
          new_v.append(v[i-1])
          new_w.append('-')
      elif (di,dj) == TOPLEFT:
          new_v.append(v[i-1])
          new_w.append(w[j-1])
      i, j = i + di, j + dj
      if (i <= 0 and j <= 0):
          break
  return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])



def global_align(v, w, delta):
  """
  Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as
  computed by traceback_global.

  :param: v
  :param: w
  :param: delta
  """
  start_time = time.time()
  tracemalloc.start()
  
  M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
  pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
  score, alignment = None, None

  # YOUR CODE HERE
  M[0][0] = 0;
  for i in range(1, len(v) + 1):
    cur_v = v[i - 1]
    M[i][0] = M[i - 1][0] + delta[cur_v]["-"]
    pointers[i][0] = UP

  for i in range(1, len(w) + 1):
    cur_w = w[i - 1]
    M[0][i] = M[0][i - 1] + delta["-"][cur_w]
    pointers[0][i] = LEFT

  for i in range(1, len(v) + 1):
    for j in range(1, len(w) + 1):
      cur_v = v[i - 1]
      cur_w = w[j - 1]

      left_score = M[i][j - 1] + delta["-"][cur_w]
      top_score = M[i - 1][j] + delta[cur_v]["-"]

      M[i][j] = max(left_score, M[i - 1][j - 1] + delta[cur_v][cur_w], top_score)

      if M[i][j] == left_score:
        pointers[i][j] = LEFT
      elif M[i][j] == top_score:
        pointers[i][j] = UP
      else:
        pointers[i][j] = TOPLEFT

  score = M[len(v)][len(w)]

  alignment = traceback_global(v,w, pointers)

  # Stop memory tracking and calculate time elapsed
  end_time = time.time()
  current, peak = tracemalloc.get_traced_memory()
  tracemalloc.stop()
  elapsed_time = end_time - start_time

  print(f"Time taken: {elapsed_time:.4f} seconds")
  print(f"Current memory usage: {current / 10**6:.2f} MB")
  print(f"Peak memory usage: {peak / 10**6:.2f} MB")

  return alignment, score, elapsed_time, current, peak

In [3]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}

global_align("TAGATA", "GTAGGCTTAAGGTTA", delta)
print("keys")

Time taken: 0.0000 seconds
Current memory usage: 0.00 MB
Peak memory usage: 0.00 MB
keys


In [46]:

import pandas as pd

def test_alignment(csv_file, delta, alignment_model):
    """
    Function to test an alignment model on a CSV file of sequences.
    
    Parameters:
    - csv_file (str): Path to the CSV file containing the sequences.
    - delta (float): Delta parameter used by the alignment model.
    - alignment_model (function): The alignment function to test (e.g., global_align).
    
    Saves the results in 'alignment_results.csv'.
    """
    # Load sequences using pandas
    df = pd.read_csv(csv_file)

    # Test on the first 10 rows
    row_list = []
    for idx, row in df.iterrows():  # 'iterrows()' provides row-wise iteration
        seq1, seq2 = row['sequence1'], row['sequence2']
        
        # Call the specified alignment model
        aligned_seq, score, elapsed_time, current_mem, peak_mem = alignment_model(seq1, seq2, delta)
        
        current_memory = current_mem / 10**6  # Convert bytes to MB
        peak_memory = peak_mem / 10**6  # Convert bytes to MB

        row_list.append([aligned_seq, score, elapsed_time, current_memory, peak_memory])

    # Save the results to a CSV
    result_df = pd.DataFrame(row_list, columns=['Aligned Seq', 'Score', 'Elapsed Time (s)', 'Current Memory (MB)', 'Peak Memory (MB)'])
    result_df.to_csv('alignment_results.csv', index=False)



In [47]:
# Chang to the desire model, testfile
test_file='dataset/mutated_sequences.csv'
model = global_align

if __name__ == "__main__":
    # Delta
    keys = ['A', 'C', 'T', 'G', '-']
    delta = {}
    for i in range(len(keys)):
        delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j] else -1 for j in range(len(keys))])}
    
    test_alignment(csv_file=test_file, delta=delta, alignment_model=model)

Time taken: 31.5258 seconds
Current memory usage: 616.10 MB
Peak memory usage: 616.18 MB


KeyboardInterrupt: 

In [95]:
#Hirschberg Implementation 
class TreeNode: #customized nodes to use in the hirschberg recursion tree 
    def __init__(self, indices, i_star, prefix_i, suffix_i, children=None):
        self.indices = indices
        self.i_star = i_star
        self.prefix_i = prefix_i
        self.suffix_i = suffix_i
        self.children = children if children is not None else []

    def add_child(self, child_node):
        self.children.append(child_node)

    def __str__(self):
        return f"{self.indices}, {self.i_star}, {self.prefix_i}, {self.suffix_i}"

#code based on sudo code given in class + example walk through!
def forward(v, w, delta):
    len_w = len(w); len_v = len(v)
    M = [[None for j in range(len_w)] for i in range(len_v)]
    M[0][0] = 0
    for i in range(1, len_v):
        M[i][0] = M[i-1][0] + delta[v[i]]['-']
    for j in range(1, len_w):
        for i in range(0, len_v):
            if (i == 0):
                M[i][j] = M[i][j-1] + delta[w[j]]['-']
            else: 
                left_score = M[i][j - 1] + delta['-'][w[j]]
                top_score = M[i-1][j] + delta[v[i]]['-']
                M[i][j] = max(left_score, M[i-1][j-1] + delta[v[i]][w[j]], top_score)
            #clear value from colum no longer in use 
            if (j >= 2):
                M[i][j-2] = None
    columns = list(zip(*M))
    return columns[len_w - 1]

def backward(v, w, delta):
    len_w = len(w); len_v = len(v)
    M = [[None for j in range(len_w)] for i in range(len_v)]
    M[-1][-1] = 0
    for i in range(len_v - 2, -1, -1):
        M[i][len_w - 1] = M[i + 1][len_w-1] + delta[v[i]]['-']
    for j in range(len_w - 2, -1, -1):
        for i in range(len_v - 1, -1, -1):
            if (i == len(v) - 1):
                M[i][j] = M[i][j+1] + delta[v[i]]['-']
            else:
                left_score = M[i][j+1] + delta['-'][w[j]]
                top_score = M[i+1][j] + delta[v[i]]['-']
                M[i][j] = max(left_score, M[i+1][j+1] + delta[v[i+1]][w[j+1]], top_score)
            #clear value from column no longer in use
            if (j + 2 < len(w)):
                M[i][j + 2] = None
    columns = list(zip(*M))
    return columns[0]

def hirschberg(v, w, delta, i, j, i_prime, j_prime):
    print("currently doing", i, j, i_prime, j_prime)
    v_adjusted = "-"+v; w_adjusted = '-'+w
    if (j_prime - j > 1):
        col_split = j_prime // 2 #round down to nearest int
        if (col_split == j): col_split = j + 1 #so suffix is never empty
        #calculating i_star
        prefixes = forward(v_adjusted[i: i_prime + 1], w_adjusted[j : col_split + 1], delta)
        suffixes = backward(v_adjusted[i: i_prime + 1], w_adjusted[col_split: j_prime + 1], delta)
        weights = [p + s for p, s in zip(prefixes, suffixes)]
        i_star = weights.index(max(weights)) 

        j_split = (int)(j+((j_prime - j)/2))
        print(i, j, i_prime, j_prime, "<-- would report")#report (but also is this where I wanna do it?)
        print(i_star + i, j_split, "SHOULD REPORT")

        left = hirschberg(v, w, delta, i, j, i_star + i, j_split)
        right = hirschberg(v, w, delta, i_star + i, j_split, i_prime, j_prime)
        print("finished", i, j, i_prime, j_prime)
        #just make them nodes here before returning them? 
        return [((i, j, i_prime, j_prime), i_star + i, prefixes[i_star], suffixes[i_star]), left, right]
    else: #base case
        print("finished", i, j, i_prime, j_prime)
        print(i, j, i_prime, j_prime, "<-- would report")
        return [(i, j, i_prime, j_prime), '-', '-', '-']

In [96]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
    
nodes = hirschberg("TG", "ATCG", delta, 0, 0, 2, 4)

currently doing 0 0 2 4
0 0 2 4 <-- would report
1 2 SHOULD REPORT
currently doing 0 0 1 2
0 0 1 2 <-- would report
0 1 SHOULD REPORT
currently doing 0 0 0 1
finished 0 0 0 1
0 0 0 1 <-- would report
currently doing 0 1 1 2
finished 0 1 1 2
0 1 1 2 <-- would report
finished 0 0 1 2
currently doing 1 2 2 4
1 2 2 4 <-- would report
1 3 SHOULD REPORT
currently doing 1 2 1 3
finished 1 2 1 3
1 2 1 3 <-- would report
currently doing 1 3 2 4
finished 1 3 2 4
1 3 2 4 <-- would report
finished 1 2 2 4
finished 0 0 2 4


In [94]:
nodes

[((0, 0, 2, 4), 1, 0, 0),
 [((0, 0, 1, 2), 0, -1, 1),
  [(0, 0, 0, 1), '-', '-', '-'],
  [(0, 1, 1, 2), '-', '-', '-']],
 [((1, 2, 2, 4), 1, -1, 1),
  [(1, 2, 1, 3), '-', '-', '-'],
  [(1, 3, 2, 4), '-', '-', '-']]]

In [9]:
#code based on sudo code found online (#https://wikidoc.org/index.php/Hirschberg's_algorithm)